In [1]:
import os
os.chdir('C:/Users/viswanath/Downloads')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [133]:
df_train = pd.read_csv('train.csv')
df_fci = pd.read_csv('fulfilment_center_info.csv')
df_meals = pd.read_csv('meal_info.csv')

In [134]:
train = pd.merge(df_train,df_fci,on='center_id')
train = pd.merge(train,df_meals,on='meal_id')

In [135]:
cols = ['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'city_code',
       'region_code', 'center_type', 'op_area', 'category', 'cuisine']

In [136]:
train['emailer_for_promotion'] = train['emailer_for_promotion'].astype(object)
train['homepage_featured'] = train['homepage_featured'].astype(object)
train['city_code'] = train['city_code'].astype(object)
train['region_code'] = train['region_code'].astype(object)
train['center_type'] = train['center_type'].astype(object)
train['op_area'] = train['op_area'].astype(object)
train = pd.get_dummies(train,columns=['emailer_for_promotion','homepage_featured','city_code','region_code','center_type','op_area','category','cuisine'])
train['center_id'] = train['center_id'].astype(object)
train['meal_id'] = train['meal_id'].astype(object)
train = pd.get_dummies(train,columns=['center_id','meal_id'])
train.shape

(456548, 247)

In [137]:
X = train.drop('num_orders',1)
y = train['num_orders']

In [138]:
df_test = pd.read_csv('test_QoiMO9B.csv')
test = pd.merge(df_test,df_fci,on='center_id')
test = pd.merge(test,df_meals,on='meal_id')
test.shape

(32573, 14)

In [139]:
test['emailer_for_promotion'] = test['emailer_for_promotion'].astype(object)
test['homepage_featured'] = test['homepage_featured'].astype(object)
test['city_code'] = test['city_code'].astype(object)
test['region_code'] = test['region_code'].astype(object)
test['center_type'] = test['center_type'].astype(object)
test['op_area'] = test['op_area'].astype(object)
test = pd.get_dummies(test,columns=['emailer_for_promotion','homepage_featured','city_code','region_code','center_type','op_area','category','cuisine'])
test['center_id'] = test['center_id'].astype(object)
test['meal_id'] = test['meal_id'].astype(object)
test = pd.get_dummies(test,columns=['center_id','meal_id'])
test.shape

(32573, 246)

In [140]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

In [141]:
def rmsle(y, y0):
    return np.sqrt(np.mean(np.square(np.log1p(y) - np.log1p(y0))))

i = 1
kf = KFold(n_splits=5,random_state=1,shuffle=True)
for train_index, test_index in kf.split(X,y):
    print('\n{} of kfold {}'.format(i,kf.n_splits))
    xtr,xvl = X.loc[train_index], X.loc[test_index]
    ytr,yvl = y[train_index], y[test_index]
    
    model = RandomForestRegressor(random_state=1)
    model.fit(xtr,ytr)
    pred_test = model.predict(xvl)
    score = rmsle(yvl,pred_test)
    print('accuracy score:', score)
    i += 1 
pred_test = model.predict(test)


1 of kfold 5
accuracy score: 0.5003456811570232

2 of kfold 5
accuracy score: 0.5027468828895985

3 of kfold 5
accuracy score: 0.5034414440787192

4 of kfold 5
accuracy score: 0.5029990706093211

5 of kfold 5
accuracy score: 0.502462334450174


In [142]:
sub = pd.read_csv('sample_submission_hSlSoT6.csv')
sub['id'] = test['id']
sub['num_orders'] = pred_test
sub['num_orders'] = sub['num_orders'].astype(int)
pd.DataFrame(sub, columns=['id','num_orders']).to_csv('last_attempt_RF.csv')